In [1]:
from typing import Dict, Text
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow_recommenders as tfrs

In [93]:
df = pd.read_csv('Dataset/fiction dataset - Sheet1.csv')
df.head(2)

,fiction_id,title,overview,language,release_date,latest_update,tags,genres,chapter
0,1,Kejahatan Cinta,"Seorang detektif swasta, Anya, sedang menyelid...",Indonesia,2023-10-01,2023-10-06,"detektif,pembunuhan,misteri,cinta,romansa","romansa,misteri",2
1,2,Raja Terakhir,"Di dunia yang dikuasai oleh iblis, seorang pem...",Indonesia,2023-10-02,2023-10-02,"fantasi,aksi,petualangan,pertarungan,kekuatan","fantasi,aksi",1


In [3]:
# read data rating
r_df = pd.read_csv('Dataset/Rating Dataset - Sheet1.csv')
rating_data = r_df[['user_id', 'fiction_id', 'rating']]
rating_data.head(3)

,user_id,fiction_id,rating
0,29,39,0
1,17,44,4
2,94,59,0


Merger

In [4]:
rating_data = rating_data.merge(df[['fiction_id', 'title', 'genres', 'overview']], left_on='fiction_id', right_on='fiction_id')
rating_data.head(3)

,user_id,fiction_id,rating,title,genres,overview
0,29,39,0,Pangeran yang Tertukar,"romansa,petualangan,aksi",Seorang gadis muda menemukan pangeran yang ter...
1,100,39,1,Pangeran yang Tertukar,"romansa,petualangan,aksi",Seorang gadis muda menemukan pangeran yang ter...
2,45,39,2,Pangeran yang Tertukar,"romansa,petualangan,aksi",Seorang gadis muda menemukan pangeran yang ter...


In [5]:
fanfic_df = df[['fiction_id', 'title', 'tags', 'genres', 'overview']]
fanfic_df.head(2)

,fiction_id,title,tags,genres,overview
0,1,Kejahatan Cinta,"detektif,pembunuhan,misteri,cinta,romansa","romansa,misteri","Seorang detektif swasta, Anya, sedang menyelid..."
1,2,Raja Terakhir,"fantasi,aksi,petualangan,pertarungan,kekuatan","fantasi,aksi","Di dunia yang dikuasai oleh iblis, seorang pem..."


In [6]:
rating_data['user_id'] = rating_data['user_id'].astype(str)

ratings = tf.data.Dataset.from_tensor_slices(dict(rating_data[['user_id', 'title', 'rating']]))
fanfics = tf.data.Dataset.from_tensor_slices(dict(fanfic_df[['title']]))

ratings = ratings.map(lambda x: {
    'title': x['title'],
    'user_id': x['user_id'],
    'rating': float(x['rating'])
})

fanfics = fanfics.map(lambda x: x['title'])

In [7]:
print('Total Data: {}'.format(len(ratings)))

Total Data: 5027


In [8]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(5027, seed=42, reshuffle_each_iteration=False)

train = ratings.take(4017)
test = ratings.skip(4017).take(1010)

In [9]:
fanfic_titles = fanfics.batch(16)
user_ids = ratings.batch(16).map(lambda x: x['user_id'])

In [10]:
unique_fanfics_titles = np.unique(np.concatenate(list(fanfic_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Fanfic: {}'.format(len(unique_fanfics_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Fanfic: 100
Unique users: 100


Note: Ada judul yang sama

In [31]:
class FanficsModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:

    super().__init__()

    embedding_dimension = 32

    # User and fanfic models.
    self.fanfic_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_fanfics_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_fanfics_titles) + 1, embedding_dimension)
    ])
    
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    


    self.rating_model = tf.keras.Sequential([
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(1),
    ])

    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanAbsoluteError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=fanfics.batch(128).map(self.fanfic_model)
        )
    )

    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features['user_id'])
    # And pick out the fanfic features and pass them into the fanfic model.
    fanfic_embeddings = self.fanfic_model(features['title'])
    
    return (
        user_embeddings,
        fanfic_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and fanfic embeddings.
        self.rating_model(
            tf.concat([user_embeddings, fanfic_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop('rating')

    user_embeddings, fanfic_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, fanfic_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [32]:
model = FanficsModel(rating_weight=0.5, retrieval_weight=0.5)
model.compile(optimizer=tf.keras.optimizers.Adagrad(1e-3))

cached_train = train.shuffle(5027).batch(32).cache()
cached_test = test.batch(32).cache()

model.fit(cached_train, epochs=5)

Epoch 1/5
126/126 [==============================] - 11s 71ms/step - mean_absolute_error: 2.3450 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0406 - factorized_top_k/top_10_categorical_accuracy: 0.0936 - factorized_top_k/top_50_categorical_accuracy: 0.4867 - factorized_top_k/top_100_categorical_accuracy: 0.9903 - loss: 59.0096 - regularization_loss: 0.0000e+00 - total_loss: 59.0096
Epoch 2/5
126/126 [==============================] - 9s 72ms/step - mean_absolute_error: 1.8638 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0416 - factorized_top_k/top_10_categorical_accuracy: 0.0951 - factorized_top_k/top_50_categorical_accuracy: 0.4927 - factorized_top_k/top_100_categorical_accuracy: 0.9903 - loss: 57.5277 - regularization_loss: 0.0000e+00 - total_loss: 57.5277
Epoch 3/5
126/126 [==============================] - 9s 68ms/step - mean_absolute_error: 1.5313 - factorize

In [74]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-10 accuracy: {metrics['factorized_top_k/top_10_categorical_accuracy']:.3f}")
print(f"Ranking MAE: {metrics['mean_absolute_error']:.3f}")

32/32 [==============================] - 2s 65ms/step - mean_absolute_error: 1.4938 - factorized_top_k/top_1_categorical_accuracy: 0.0069 - factorized_top_k/top_5_categorical_accuracy: 0.0535 - factorized_top_k/top_10_categorical_accuracy: 0.1030 - factorized_top_k/top_50_categorical_accuracy: 0.5069 - factorized_top_k/top_100_categorical_accuracy: 0.9970 - loss: 55.1169 - regularization_loss: 0.0000e+00 - total_loss: 55.1169

Retrieval top-10 accuracy: 0.103
Ranking MAE: 1.494


In [78]:
def predict_rating(user, fanfic):
    trained_fanfic_embeddings, trained_user_embeddings, predicted_rating = model({
        "user_id": np.array([str(user)]),
        "title": np.array([fanfic])
    })
    return fanfic, predicted_rating.numpy()[0][0]

In [86]:
def predict_fanfic(user, top_n=10):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # Recommends fanfics out of the entire fanfics dataset.
    index.index_from_dataset(
        tf.data.Dataset.zip((fanfics.batch(100), fanfics.batch(100).map(model.fanfic_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    
    # Print the number of recommendations obtained.
    num_recommendations = titles.shape[1]
    print(f"Number of Recommendations Obtained: {num_recommendations}")

    # Extract recommended titles, ensuring that the number of recommendations is at least `top_n`.
    recommended_titles = [title.decode('utf-8') for title in titles[0, :max(top_n, num_recommendations)].numpy()]

    # Merge with fanfic_df to get genre information.
    recommendations_with_genre = pd.merge(
        pd.DataFrame({'title': recommended_titles}),
        fanfic_df[['title', 'genres']],
        on='title',
        how='left'
    )

    # Predict and store ratings for the recommended titles.
    predictions = []
    for title, genre in zip(recommendations_with_genre['title'], recommendations_with_genre['genres']):
        fanfic, predicted_rating = predict_rating(user, title)
        predictions.append((fanfic, genre, predicted_rating))

    # Sort recommendations based on predicted ratings.
    sorted_predictions = sorted(predictions, key=lambda x: x[2], reverse=True)

    print(f'Top {top_n} recommendations for user {user}:\n')
    for i, (title, genre, predicted_rating) in enumerate(sorted_predictions[:top_n]):
        # Print sorted recommendation with genre and rating.
        print(f'{i + 1}. {title} (Genre: {genre}) - Predicted Rating: {predicted_rating:.2f}')

# Example usage
predict_fanfic(50)

Number of Recommendations Obtained: 10
Top 10 recommendations for user 50:

1. Ratu Badai (Genre: fantasi,petualangan) - Predicted Rating: 2.58
2. Kembali ke Masa Depan (Genre: sci-fi) - Predicted Rating: 2.57
3. Ksatria Elang: Darius (Genre: aksi,fantasi) - Predicted Rating: 2.57
4. Petualangan di Dunia Fantasi (Genre: petualangan,fantasi) - Predicted Rating: 2.57
5. Cinta di Balik Layar (Genre: romansa,drama) - Predicted Rating: 2.56
6. The Last Dragon (Genre: aksi,fantasi) - Predicted Rating: 2.54
7. Keajaiban Cinta (Genre: romansa,drama) - Predicted Rating: 2.52
8. Pertarungan Akhir (Genre: aksi,petualangan) - Predicted Rating: 2.52
9. Rahasia Pulau Terpencil (Genre: horor,misteri) - Predicted Rating: 2.48
10. Cinta Sejati (Genre: drama,romansa) - Predicted Rating: 2.46
